# Full Year 2018 - Analysis of Transient ADR

#### Compare Transient ADR and Group ADR - Is there a relationship?

In [275]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from shapely.geometry import Point
import calendar
import datetime

In [276]:
jan_18 = pd.read_excel('data/Groups360_201801.xls')
feb_18 = pd.read_excel('data/Groups360_201802.xls')
mar_18 = pd.read_excel('data/Groups360_201803.xls')
apr_18 = pd.read_excel('data/Groups360_201804.xls')
may_18 = pd.read_excel('data/Groups360_201805.xls')
jun_18 = pd.read_excel('data/Groups360_201806.xls')
jul_18 = pd.read_excel('data/Groups360_201807.xls')
aug_18 = pd.read_excel('data/Groups360_201808.xls')
sep_18 = pd.read_excel('data/Groups360_201809.xls')
oct_18 = pd.read_excel('data/Groups360_201810.xls')
nov_18 = pd.read_excel('data/Groups360_201811.xls')
dec_18 = pd.read_excel('data/Groups360_201812.xls')

#### Confirm dataframes are equivalent size before merging or concatenating

In [277]:
jan_18.shape

(1829, 50)

In [278]:
feb_18.shape

(1652, 50)

In [279]:
mar_18.shape

(1833, 50)

In [280]:
apr_18.shape

(1770, 50)

In [281]:
may_18.shape

(1829, 50)

In [282]:
jun_18.shape

(1770, 50)

In [283]:
jul_18.shape

(1829, 50)

In [284]:
aug_18.shape

(1829, 50)

In [285]:
sep_18.shape

(1770, 50)

In [286]:
oct_18.shape

(1829, 50)

In [287]:
nov_18.shape

(1770, 50)

In [288]:
dec_18.shape

(1829, 50)

In [289]:
fy_2018 = pd.concat([jan_18, feb_18, mar_18, apr_18, may_18, jun_18, jul_18, aug_18, sep_18, oct_18, nov_18, dec_18], 
                    sort=False)
fy_2018.shape

(21539, 51)

In [290]:
fy_2018.head()

,SegmentName,Region,Lat,Lng,State,State Formula,Date,SupplyTY,Supply%Chg,DemandTY,...,TransientOcc%Chg,TransientADRTY,TransientADR%Chg,TransientRevPARTY,TransientRevPAR%Chg,SampleProps,CensusProps,SampleRooms,CensusRooms,State formula
0,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180101.0,15383.0,1.8,8483.0,...,24.6,198.32,19.4,92.00,48.8,46.0,52.0,15165.0,15383.0,NaN
1,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180102.0,15383.0,1.8,8998.0,...,16.7,175.86,10.9,85.92,29.4,46.0,52.0,15165.0,15383.0,NaN
2,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180103.0,15383.0,1.8,10010.0,...,18.1,176.12,11.4,91.24,31.6,46.0,52.0,15165.0,15383.0,NaN
3,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180104.0,15383.0,1.8,10187.0,...,43.8,175.49,3.9,89.95,49.4,46.0,52.0,15165.0,15383.0,NaN
4,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180105.0,15383.0,1.8,9779.0,...,47.9,169.88,-0.1,83.51,47.7,46.0,52.0,15165.0,15383.0,NaN


#### Drop the duplicative columns used to extract the state from the region

In [291]:
fy_2018 = fy_2018.drop(['State Formula', 'State formula'], axis=1)
fy_2018.shape

(21539, 49)

In [292]:
fy_2018.head()

,SegmentName,Region,Lat,Lng,State,Date,SupplyTY,Supply%Chg,DemandTY,Demand%Chg,...,TransientOccTY,TransientOcc%Chg,TransientADRTY,TransientADR%Chg,TransientRevPARTY,TransientRevPAR%Chg,SampleProps,CensusProps,SampleRooms,CensusRooms
0,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,20180101.0,15383.0,1.8,8483.0,19.0,...,46.4,24.6,198.32,19.4,92.00,48.8,46.0,52.0,15165.0,15383.0
1,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,20180102.0,15383.0,1.8,8998.0,15.2,...,48.9,16.7,175.86,10.9,85.92,29.4,46.0,52.0,15165.0,15383.0
2,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,20180103.0,15383.0,1.8,10010.0,8.0,...,51.8,18.1,176.12,11.4,91.24,31.6,46.0,52.0,15165.0,15383.0
3,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,20180104.0,15383.0,1.8,10187.0,6.0,...,51.3,43.8,175.49,3.9,89.95,49.4,46.0,52.0,15165.0,15383.0
4,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,20180105.0,15383.0,1.8,9779.0,-2.1,...,49.2,47.9,169.88,-0.1,83.51,47.7,46.0,52.0,15165.0,15383.0


In [293]:
fy_2018['Date'] = fy_2018['Date'].apply(lambda x:  pd.to_datetime(str(x), format='%Y%m%d'))

In [294]:
fy_2018.Date.dtype

dtype('<M8[ns]')

In [295]:
fy_2018.columns.values

array(['SegmentName', 'Region', 'Lat', 'Lng', 'State', 'Date', 'SupplyTY',
       'Supply%Chg', 'DemandTY', 'Demand%Chg', 'RevenueTY', 'Rev%Chg',
       'OccTY', 'Occ%Chg', 'ADRTY', 'ADR%Chg', 'RevParTY', 'RevPar%Chg',
       'GroupDemandTY', 'GroupRevenueTY', 'GroupRev%Chg', 'GroupOccTY',
       'GroupOcc%Chg', 'GroupADRTY', 'GroupADR%Chg', 'GroupRevPARTY',
       'GroupRevPAR%Chg', 'ContractDemandTY', 'ContractRevenueTY',
       'ContractRev%Chg', 'ContractOccTY', 'ContractOcc%Chg',
       'ContractADRTY', 'ContractADR%Chg', 'ContractRevPARTY',
       'ContractRevPAR%Chg', 'TransientDemandTY', 'TransientRevenueTY',
       'TransientRev%Chg', 'TransientOccTY', 'TransientOcc%Chg',
       'TransientADRTY', 'TransientADR%Chg', 'TransientRevPARTY',
       'TransientRevPAR%Chg', 'SampleProps', 'CensusProps', 'SampleRooms',
       'CensusRooms'], dtype=object)

   #### Aggregate Group Revenue TY and Group Demand TY by Month and Region         

In [296]:
fy_2018_1 = fy_2018.groupby(['Region', 'Date', 'Lat', 'Lng'])['GroupDemandTY','GroupRevenueTY', 
                                                               'TransientDemandTY', 'TransientRevenueTY'].sum().reset_index()
fy_2018_1.head()

,Region,Date,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY
0,"Anaheim/Santa Ana, CA",2018-01-01,33.834197,-117.985579,772.0,253539.0,7136.0,1415207.0
1,"Anaheim/Santa Ana, CA",2018-01-02,33.834197,-117.985579,905.0,127444.0,7516.0,1321768.0
2,"Anaheim/Santa Ana, CA",2018-01-03,33.834197,-117.985579,1435.0,215608.0,7969.0,1403525.0
3,"Anaheim/Santa Ana, CA",2018-01-04,33.834197,-117.985579,1691.0,255734.0,7885.0,1383715.0
4,"Anaheim/Santa Ana, CA",2018-01-05,33.834197,-117.985579,1611.0,257831.0,7562.0,1284618.0


#### Parse date and month from date column

In [297]:
#Full Month name = dt.month_name()
fy_2018_1['Year'] = fy_2018_1['Date'].dt.year
fy_2018_1['Month'] = fy_2018_1['Date'].dt.month
fy_2018_1.head()

,Region,Date,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,Year,Month
0,"Anaheim/Santa Ana, CA",2018-01-01,33.834197,-117.985579,772.0,253539.0,7136.0,1415207.0,2018,1
1,"Anaheim/Santa Ana, CA",2018-01-02,33.834197,-117.985579,905.0,127444.0,7516.0,1321768.0,2018,1
2,"Anaheim/Santa Ana, CA",2018-01-03,33.834197,-117.985579,1435.0,215608.0,7969.0,1403525.0,2018,1
3,"Anaheim/Santa Ana, CA",2018-01-04,33.834197,-117.985579,1691.0,255734.0,7885.0,1383715.0,2018,1
4,"Anaheim/Santa Ana, CA",2018-01-05,33.834197,-117.985579,1611.0,257831.0,7562.0,1284618.0,2018,1


In [298]:
fy_2018_1.tail()

,Region,Date,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,Year,Month
21530,"West Palm Beach/Boca Raton, FL",2018-12-27,26.709723,-80.064163,509.0,98981.0,5594.0,3258240.0,2018,12
21531,"West Palm Beach/Boca Raton, FL",2018-12-28,26.709723,-80.064163,600.0,111701.0,5843.0,3484511.0,2018,12
21532,"West Palm Beach/Boca Raton, FL",2018-12-29,26.709723,-80.064163,626.0,118179.0,5921.0,3544871.0,2018,12
21533,"West Palm Beach/Boca Raton, FL",2018-12-30,26.709723,-80.064163,429.0,80978.0,5391.0,3351651.0,2018,12
21534,"West Palm Beach/Boca Raton, FL",2018-12-31,26.709723,-80.064163,370.0,72809.0,5609.0,3373862.0,2018,12


#### Aggregate data by Month

In [299]:
fy_2018_2 = fy_2018_1.groupby(['Year', 'Month', 'Region', 'Lat', 'Lng'])['GroupDemandTY','GroupRevenueTY', 
                                'TransientDemandTY', 'TransientRevenueTY'].sum().reset_index()
fy_2018_2.tail()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY
703,2018,12,"Seattle, WA",47.608013,-122.335167,50849.0,8792249.0,261537.0,47708640.0
704,2018,12,"St Louis, MO-IL",38.627003,-90.199402,37759.0,4402890.0,92468.0,12543277.0
705,2018,12,"Tampa/St Petersburg, FL",27.964157,-82.452606,67511.0,10073333.0,178925.0,32834933.0
706,2018,12,"Washington, DC-MD-VA",38.893513,-77.154663,191215.0,35709491.0,599578.0,101615277.0
707,2018,12,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,45335.0,9734761.0,111133.0,43431407.0


#### Calculate the Group ADR and Transient ADR by Region for the month

In [300]:
fy_2018_2['GroupADR'] = (fy_2018_2.GroupRevenueTY / fy_2018_2.GroupDemandTY).round(2)
fy_2018_2['TransADR'] = (fy_2018_2.TransientRevenueTY / fy_2018_2.TransientDemandTY).round(2)
fy_2018_2.tail()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR
703,2018,12,"Seattle, WA",47.608013,-122.335167,50849.0,8792249.0,261537.0,47708640.0,172.91,182.42
704,2018,12,"St Louis, MO-IL",38.627003,-90.199402,37759.0,4402890.0,92468.0,12543277.0,116.61,135.65
705,2018,12,"Tampa/St Petersburg, FL",27.964157,-82.452606,67511.0,10073333.0,178925.0,32834933.0,149.21,183.51
706,2018,12,"Washington, DC-MD-VA",38.893513,-77.154663,191215.0,35709491.0,599578.0,101615277.0,186.75,169.48
707,2018,12,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,45335.0,9734761.0,111133.0,43431407.0,214.73,390.81


#### Rank the Group ADR and the Transient ADR in descending order to show the months with the highest GroupADR and Transient (Leisure) ADR

In [301]:
fy_2018_2['GroupRank'] = fy_2018_2.groupby('Month')['GroupADR'].rank(ascending = False)  
fy_2018_2['TransRank'] = fy_2018_2.groupby('Month')['TransADR'].rank(ascending = False)
fy_2018_2.head(10)

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank
0,2018,1,"Anaheim/Santa Ana, CA",33.834197,-117.985579,123337.0,27156640.0,192850.0,38785443.0,220.18,201.12,15.0,20.0
1,2018,1,"Atlanta, GA",33.767634,-84.560691,266179.0,48225171.0,277693.0,53655729.0,181.18,193.22,28.0,22.0
2,2018,1,"Austin, TX",30.307462,-98.033602,101873.0,20593488.0,103502.0,20988844.0,202.15,202.79,19.0,18.0
3,2018,1,"Baltimore, MD",39.299236,-76.609383,40616.0,5476141.0,77929.0,10685128.0,134.83,137.11,45.0,51.0
4,2018,1,"Birmingham, AL",33.543682,-86.779633,18656.0,2470727.0,29439.0,4217082.0,132.44,143.25,47.0,47.0
5,2018,1,"Boston, MA",42.361145,-71.057083,99398.0,17518952.0,292851.0,53300810.0,176.25,182.01,31.0,27.0
6,2018,1,California Central Coast,35.315787,-120.857870,68965.0,12169253.0,124568.0,27041348.0,176.46,217.08,30.0,16.0
7,2018,1,"Charleston, SC",32.776566,-79.930923,31605.0,5452018.0,45395.0,7855755.0,172.50,173.05,34.0,33.0
8,2018,1,"Charlotte, NC-SC",35.227085,-80.843124,50792.0,8728029.0,64959.0,12403564.0,171.84,190.94,35.0,23.0
9,2018,1,"Chattanooga, TN-GA",35.045631,-85.309677,7766.0,925329.0,16725.0,2075349.0,119.15,124.09,52.0,53.0


In [351]:
fy_2018_2.to_csv('TransientADR/fy_2018_2.csv', index = False)

#### Sort the Transient ADR from highest (rank of 1.0) to lowest

In [303]:
## Could not determine how to apply this code to my Month column and sort the months in chron order.
#months_order = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November"
 #              "December"]
#cat = pd.Categorical(["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November"
 #              "December"], 
  #                   categories=months_order, ordered=True)

In [304]:
fy_2018_2 = fy_2018_2.sort_values(['Month','TransRank'], axis=0, ascending=[True,True])
fy_2018_2.head(10)

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank
13,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,121534.0,67579065.0,362.98,556.05,2.0,1.0
58,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,98937.0,35468946.0,274.10,358.50,6.0,2.0
32,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,357073.0,125355418.0,299.84,351.06,5.0,3.0
19,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,54846.0,19050594.0,315.15,347.35,3.0,4.0
18,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,67878.0,22973662.0,249.44,338.46,9.0,5.0
50,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,446435.0,141978303.0,403.85,318.03,1.0,6.0
39,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,362118.0,110551026.0,305.40,305.29,4.0,7.0
52,2018,1,"Sarasota/Bradenton, FL",27.341274,-82.528267,11632.0,2339502.0,22331.0,6128261.0,201.13,274.43,20.0,8.0
22,2018,1,"Fort Myers, FL",26.640629,-81.872307,29748.0,6737611.0,42250.0,11499173.0,226.49,272.17,12.0,9.0
30,2018,1,"Los Angeles/Long Beach, CA",34.052235,-118.243683,231908.0,55379695.0,624488.0,161990527.0,238.80,259.40,11.0,10.0


#### Is there a correlation between GroupADR and Transient ADR?  Evaluate by month.

In [346]:
month_rates_corr = fy_2018_2.groupby('Month')['GroupADR'].corr(fy_2018_2['TransADR']).reset_index().round(2)
month_rates_corr

,Month,GroupADR
0,1,0.89
1,2,0.94
2,3,0.91
3,4,0.91
4,5,0.87
5,6,0.94
6,7,0.95
7,8,0.96
8,9,0.93
9,10,0.95


In [347]:
month_rates_corr.to_csv('TransientADR/month_rates_corr.csv', index = False)

In [358]:
ca = fy_2018_2.loc[fy_2018_2.Region == 'Anaheim/Santa Ana, CA']
ca.GroupADR.corr(ca.TransADR)

0.06883782798864822

In [360]:
ca

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl
0,2018,1,"Anaheim/Santa Ana, CA",33.834197,-117.985579,123337.0,27156640.0,192850.0,38785443.0,220.18,201.12,15.0,20.0,NaN
59,2018,2,"Anaheim/Santa Ana, CA",33.834197,-117.985579,130818.0,29171948.0,195437.0,41844406.0,223.00,214.11,18.0,22.0,NaN
118,2018,3,"Anaheim/Santa Ana, CA",33.834197,-117.985579,127508.0,29520134.0,245769.0,55554336.0,231.52,226.04,22.0,26.0,NaN
177,2018,4,"Anaheim/Santa Ana, CA",33.834197,-117.985579,130578.0,29216555.0,229204.0,48795111.0,223.75,212.89,20.0,28.0,NaN
236,2018,5,"Anaheim/Santa Ana, CA",33.834197,-117.985579,119636.0,27730005.0,231425.0,48661628.0,231.79,210.27,14.0,26.0,NaN
295,2018,6,"Anaheim/Santa Ana, CA",33.834197,-117.985579,125501.0,28759050.0,257578.0,59024610.0,229.15,229.15,14.0,19.0,NaN
354,2018,7,"Anaheim/Santa Ana, CA",33.834197,-117.985579,103889.0,23137457.0,302250.0,84278742.0,222.71,278.84,13.0,11.0,NaN
413,2018,8,"Anaheim/Santa Ana, CA",33.834197,-117.985579,97146.0,20623167.0,290530.0,77605834.0,212.29,267.12,11.0,10.0,NaN
472,2018,9,"Anaheim/Santa Ana, CA",33.834197,-117.985579,110490.0,26138347.0,231926.0,50233885.0,236.57,216.59,9.0,16.0,NaN
531,2018,10,"Anaheim/Santa Ana, CA",33.834197,-117.985579,133875.0,31346532.0,244953.0,52028245.0,234.15,212.40,15.0,21.0,NaN


In [359]:
ga = fy_2018_2.loc[fy_2018_2.Region == 'Atlanta, GA']
ga.GroupADR.corr(ga.TransADR)

0.9020436037373294

In [348]:
region_rates_corr = fy_2018_2.groupby('Region')['GroupADR'].corr(fy_2018_2['TransADR']).reset_index().round(2)
region_rates_corr

,Region,GroupADR
0,"Anaheim/Santa Ana, CA",0.07
1,"Atlanta, GA",0.90
2,"Austin, TX",0.91
3,"Baltimore, MD",0.92
4,"Birmingham, AL",0.72
5,"Boston, MA",0.95
6,California Central Coast,0.96
7,"Charleston, SC",0.92
8,"Charlotte, NC-SC",0.87
9,"Chattanooga, TN-GA",0.66


In [349]:
region_rates_corr.to_csv('TransientADR/region_rates_corr.csv', index = False)

# After calculating for each month, add a constant and enter the correlation

In [307]:
#jan_18_agg2['ADR Correl'] = (jan_18_agg2['GroupADR'].corr(jan_18_agg2['TransADR'])*100).round(2)
fy_2018_2['ADR Correl'] = (fy_2018_2.groupby('Month')['GroupADR'].corr(fy_2018_2['TransADR'])*100).round(2)
fy_2018_2.head()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl
13,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,121534.0,67579065.0,362.98,556.05,2.0,1.0,NaN
58,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,98937.0,35468946.0,274.10,358.50,6.0,2.0,NaN
32,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,357073.0,125355418.0,299.84,351.06,5.0,3.0,NaN
19,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,54846.0,19050594.0,315.15,347.35,3.0,4.0,NaN
18,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,67878.0,22973662.0,249.44,338.46,9.0,5.0,NaN


In [308]:
fy_2018_2.shape

(708, 14)

#### Check for null values

In [309]:
fy_2018_2.isnull()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl
13,False,False,False,False,False,False,False,False,False,False,False,False,False,True
58,False,False,False,False,False,False,False,False,False,False,False,False,False,True
32,False,False,False,False,False,False,False,False,False,False,False,False,False,True
19,False,False,False,False,False,False,False,False,False,False,False,False,False,True
18,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,False,False,False,False,False,False,False,False,False,False,False,False,False,True
663,False,False,False,False,False,False,False,False,False,False,False,False,False,True
683,False,False,False,False,False,False,False,False,False,False,False,False,False,True
669,False,False,False,False,False,False,False,False,False,True,True,True,True,True


#### Drop the NaN Values

In [310]:
fy_2018_tadr = fy_2018_2[fy_2018_2['GroupADR'].notnull()]
fy_2018_tadr.tail()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl
704,2018,12,"St Louis, MO-IL",38.627003,-90.199402,37759.0,4402890.0,92468.0,12543277.0,116.61,135.65,51.0,53.0,NaN
660,2018,12,"Cincinnati, OH-KY-IN",39.103119,-84.512016,18078.0,2246499.0,57601.0,7788208.0,124.27,135.21,50.0,54.0,NaN
678,2018,12,"Las Vegas, NV",36.114647,-115.172813,109036.0,11973458.0,678564.0,83544938.0,109.81,123.12,55.0,55.0,NaN
663,2018,12,"Colorado Springs, CO",38.846127,-104.800644,7877.0,896504.0,22253.0,2599979.0,113.81,116.84,53.0,56.0,NaN
683,2018,12,"Myrtle Beach, SC",33.689060,-78.886696,19468.0,1632746.0,22306.0,2436703.0,83.87,109.24,57.0,57.0,NaN


In [311]:
fy_2018_tadr.shape

(690, 14)

#### Sort the DF by Month and Transient (Leisure) Rank (1 = high); the Group ADR Rank can then be compared to the Transient Rank.

In [312]:
fy_2018_ranked = fy_2018_tadr.sort_values(['Month','TransRank'], axis=0, ascending=[True,True])
fy_2018_ranked.head(10)

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl
13,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,121534.0,67579065.0,362.98,556.05,2.0,1.0,NaN
58,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,98937.0,35468946.0,274.10,358.50,6.0,2.0,NaN
32,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,357073.0,125355418.0,299.84,351.06,5.0,3.0,NaN
19,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,54846.0,19050594.0,315.15,347.35,3.0,4.0,NaN
18,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,67878.0,22973662.0,249.44,338.46,9.0,5.0,NaN
50,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,446435.0,141978303.0,403.85,318.03,1.0,6.0,NaN
39,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,362118.0,110551026.0,305.40,305.29,4.0,7.0,NaN
52,2018,1,"Sarasota/Bradenton, FL",27.341274,-82.528267,11632.0,2339502.0,22331.0,6128261.0,201.13,274.43,20.0,8.0,NaN
22,2018,1,"Fort Myers, FL",26.640629,-81.872307,29748.0,6737611.0,42250.0,11499173.0,226.49,272.17,12.0,9.0,NaN
30,2018,1,"Los Angeles/Long Beach, CA",34.052235,-118.243683,231908.0,55379695.0,624488.0,161990527.0,238.80,259.40,11.0,10.0,NaN


In [313]:
fy_2018_ranked.tail(10)

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl
652,2018,12,"Baltimore, MD",39.299236,-76.609383,28860.0,3782119.0,78809.0,11138077.0,131.05,141.33,48.0,48.0,NaN
658,2018,12,"Chattanooga, TN-GA",35.045631,-85.309677,4576.0,522940.0,15593.0,2201146.0,114.28,141.16,52.0,49.0,NaN
680,2018,12,"Louisville, KY-IN",38.328732,-85.764771,15354.0,2117399.0,31711.0,4456571.0,137.91,140.54,45.0,50.0,NaN
661,2018,12,"Cleveland, OH",41.505493,-81.681290,11109.0,1562995.0,57099.0,7935228.0,140.70,138.97,42.0,51.0,NaN
653,2018,12,"Birmingham, AL",33.543682,-86.779633,10716.0,1410476.0,32934.0,4495938.0,131.62,136.51,47.0,52.0,NaN
704,2018,12,"St Louis, MO-IL",38.627003,-90.199402,37759.0,4402890.0,92468.0,12543277.0,116.61,135.65,51.0,53.0,NaN
660,2018,12,"Cincinnati, OH-KY-IN",39.103119,-84.512016,18078.0,2246499.0,57601.0,7788208.0,124.27,135.21,50.0,54.0,NaN
678,2018,12,"Las Vegas, NV",36.114647,-115.172813,109036.0,11973458.0,678564.0,83544938.0,109.81,123.12,55.0,55.0,NaN
663,2018,12,"Colorado Springs, CO",38.846127,-104.800644,7877.0,896504.0,22253.0,2599979.0,113.81,116.84,53.0,56.0,NaN
683,2018,12,"Myrtle Beach, SC",33.689060,-78.886696,19468.0,1632746.0,22306.0,2436703.0,83.87,109.24,57.0,57.0,NaN


In [314]:
fy_2018_ranked['GroupADR Ratio TransADR'] = (fy_2018_ranked['GroupADR'] / fy_2018_ranked['TransADR']*100).round(2)
fy_2018_ranked.tail(15)

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR
659,2018,12,"Chicago, IL",41.881832,-87.623177,159340.0,23265714.0,591498.0,88242146.0,146.01,149.18,38.0,43.0,NaN,97.88
694,2018,12,"Portland, OR",45.523064,-122.676483,13678.0,2064503.0,114192.0,16931637.0,150.94,148.27,33.0,44.0,NaN,101.80
676,2018,12,"Kansas City, MO-KS",39.099724,-94.578331,27260.0,3612034.0,77644.0,11418761.0,132.50,147.07,46.0,45.0,NaN,90.09
687,2018,12,"Norfolk/Virginia Beach, VA",36.850769,-76.285873,16080.0,1812613.0,64787.0,9262042.0,112.72,142.96,54.0,46.0,NaN,78.85
682,2018,12,"Minneapolis/St Paul, MN-WI",44.986656,-93.258133,38854.0,5061262.0,140542.0,19889235.0,130.26,141.52,49.0,47.0,NaN,92.04
652,2018,12,"Baltimore, MD",39.299236,-76.609383,28860.0,3782119.0,78809.0,11138077.0,131.05,141.33,48.0,48.0,NaN,92.73
658,2018,12,"Chattanooga, TN-GA",35.045631,-85.309677,4576.0,522940.0,15593.0,2201146.0,114.28,141.16,52.0,49.0,NaN,80.96
680,2018,12,"Louisville, KY-IN",38.328732,-85.764771,15354.0,2117399.0,31711.0,4456571.0,137.91,140.54,45.0,50.0,NaN,98.13
661,2018,12,"Cleveland, OH",41.505493,-81.681290,11109.0,1562995.0,57099.0,7935228.0,140.70,138.97,42.0,51.0,NaN,101.24
653,2018,12,"Birmingham, AL",33.543682,-86.779633,10716.0,1410476.0,32934.0,4495938.0,131.62,136.51,47.0,52.0,NaN,96.42


In [315]:
fy_2018_ranked.to_csv('TransientADR/fy_2018_ranked.csv', index = False)

In [316]:
#Drop Rows with Hawaii to allow for a map visualization
fy_2018_ranked_conus = fy_2018_ranked.loc[fy_2018_ranked['Region'] != 'Oahu Island, HI']
fy_2018_ranked_conus

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR
13,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,121534.0,67579065.0,362.98,556.05,2.0,1.0,NaN,65.28
58,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,98937.0,35468946.0,274.10,358.50,6.0,2.0,NaN,76.46
32,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,357073.0,125355418.0,299.84,351.06,5.0,3.0,NaN,85.41
19,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,54846.0,19050594.0,315.15,347.35,3.0,4.0,NaN,90.73
18,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,67878.0,22973662.0,249.44,338.46,9.0,5.0,NaN,73.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,2018,12,"St Louis, MO-IL",38.627003,-90.199402,37759.0,4402890.0,92468.0,12543277.0,116.61,135.65,51.0,53.0,NaN,85.96
660,2018,12,"Cincinnati, OH-KY-IN",39.103119,-84.512016,18078.0,2246499.0,57601.0,7788208.0,124.27,135.21,50.0,54.0,NaN,91.91
678,2018,12,"Las Vegas, NV",36.114647,-115.172813,109036.0,11973458.0,678564.0,83544938.0,109.81,123.12,55.0,55.0,NaN,89.19
663,2018,12,"Colorado Springs, CO",38.846127,-104.800644,7877.0,896504.0,22253.0,2599979.0,113.81,116.84,53.0,56.0,NaN,97.41


In [317]:
fy_2018_ranked_conus.to_csv('TransientADR/fy_2018_ranked_conus.csv', index = False)

In [318]:
#Top Leisure Rate Regions
top_leisure_rate = fy_2018_ranked.nlargest(12,'TransADR')
top_leisure_rate

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR
662,2018,12,Colorado Area,39.482231,-106.046181,25303.0,5421703.0,154041.0,85664473.0,214.27,556.11,7.0,1.0,NaN,38.53
13,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,121534.0,67579065.0,362.98,556.05,2.0,1.0,NaN,65.28
72,2018,2,Colorado Area,39.482231,-106.046181,52475.0,20207026.0,133296.0,71685551.0,385.08,537.79,1.0,1.0,NaN,71.60
131,2018,3,Colorado Area,39.482231,-106.046181,46934.0,18417527.0,157560.0,77975202.0,392.41,494.89,1.0,1.0,NaN,79.29
176,2018,3,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,62015.0,19586339.0,121979.0,55316288.0,315.83,453.49,4.0,2.0,NaN,69.64
668,2018,12,Florida Keys,24.778338,-80.921608,9050.0,2498466.0,66934.0,29712184.0,276.07,443.90,3.0,2.0,NaN,62.19
137,2018,3,Florida Keys,24.778338,-80.921608,10385.0,3901736.0,84264.0,36957992.0,375.71,438.60,2.0,3.0,NaN,85.66
117,2018,2,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,56990.0,18937067.0,107789.0,45971448.0,332.29,426.49,3.0,2.0,NaN,77.91
568,2018,10,"New York, NY",40.730610,-73.935242,260592.0,100307576.0,1049406.0,446113388.0,384.92,425.11,1.0,1.0,NaN,90.55
681,2018,12,"Miami/Hialeah, FL",25.761681,-80.191788,89276.0,25756101.0,433413.0,183995791.0,288.50,424.53,2.0,3.0,NaN,67.96


In [319]:
top_leisure_rate['Counts'] = top_leisure_rate.groupby(['Region'])['TransADR'].transform('count')
top_leisure_rate

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR,Counts
662,2018,12,Colorado Area,39.482231,-106.046181,25303.0,5421703.0,154041.0,85664473.0,214.27,556.11,7.0,1.0,NaN,38.53,4
13,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,121534.0,67579065.0,362.98,556.05,2.0,1.0,NaN,65.28,4
72,2018,2,Colorado Area,39.482231,-106.046181,52475.0,20207026.0,133296.0,71685551.0,385.08,537.79,1.0,1.0,NaN,71.60,4
131,2018,3,Colorado Area,39.482231,-106.046181,46934.0,18417527.0,157560.0,77975202.0,392.41,494.89,1.0,1.0,NaN,79.29,4
176,2018,3,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,62015.0,19586339.0,121979.0,55316288.0,315.83,453.49,4.0,2.0,NaN,69.64,2
668,2018,12,Florida Keys,24.778338,-80.921608,9050.0,2498466.0,66934.0,29712184.0,276.07,443.90,3.0,2.0,NaN,62.19,2
137,2018,3,Florida Keys,24.778338,-80.921608,10385.0,3901736.0,84264.0,36957992.0,375.71,438.60,2.0,3.0,NaN,85.66,2
117,2018,2,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,56990.0,18937067.0,107789.0,45971448.0,332.29,426.49,3.0,2.0,NaN,77.91,2
568,2018,10,"New York, NY",40.730610,-73.935242,260592.0,100307576.0,1049406.0,446113388.0,384.92,425.11,1.0,1.0,NaN,90.55,1
681,2018,12,"Miami/Hialeah, FL",25.761681,-80.191788,89276.0,25756101.0,433413.0,183995791.0,288.50,424.53,2.0,3.0,NaN,67.96,1


In [321]:
top_leisure_rate.to_csv('TransientADR/top_ten_leisure.csv', index = False)

In [322]:
#Top Group Rates by Region
top_group_rates = fy_2018_ranked.nlargest(12,'GroupADR')
top_group_rates

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR
509,2018,9,"New York, NY",40.730610,-73.935242,261257.0,129983502.0,941925.0,383202930.0,497.53,406.83,1.0,1.0,NaN,122.29
686,2018,12,"New York, NY",40.730610,-73.935242,132260.0,57973317.0,1126797.0,462549234.0,438.33,410.50,1.0,4.0,NaN,106.78
50,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,446435.0,141978303.0,403.85,318.03,1.0,6.0,NaN,126.98
131,2018,3,Colorado Area,39.482231,-106.046181,46934.0,18417527.0,157560.0,77975202.0,392.41,494.89,1.0,1.0,NaN,79.29
72,2018,2,Colorado Area,39.482231,-106.046181,52475.0,20207026.0,133296.0,71685551.0,385.08,537.79,1.0,1.0,NaN,71.60
568,2018,10,"New York, NY",40.730610,-73.935242,260592.0,100307576.0,1049406.0,446113388.0,384.92,425.11,1.0,1.0,NaN,90.55
627,2018,11,"New York, NY",40.730610,-73.935242,210734.0,79643797.0,1013350.0,402573861.0,377.94,397.27,1.0,1.0,NaN,95.13
137,2018,3,Florida Keys,24.778338,-80.921608,10385.0,3901736.0,84264.0,36957992.0,375.71,438.60,2.0,3.0,NaN,85.66
267,2018,5,"Louisville, KY-IN",38.328732,-85.764771,33560.0,12549900.0,41829.0,9752480.0,373.95,233.15,1.0,15.0,NaN,160.39
155,2018,3,"New York, NY",40.730610,-73.935242,248712.0,91629028.0,968282.0,301307877.0,368.41,311.18,3.0,8.0,NaN,118.39


In [323]:
top_group_rates['Counts'] = top_group_rates.groupby('Region')['GroupADR'].transform('count')

In [324]:
top_group_rates['Region'].value_counts()

New York, NY                   5
Colorado Area                  3
Florida Keys                   2
Louisville, KY-IN              1
San Francisco/San Mateo, CA    1
Name: Region, dtype: int64

In [325]:
top_group_rates.to_csv('TransientADR/top_group_rate.csv', index = False)

#### Alternative code to select top leisure rates for the year

In [326]:
#Alternative to getting the value counts of the regions with the highest transient ADR
top_ten_leisure = top_ten_trans_adr['Region'].value_counts()
top_ten_leisure = top_ten_leisure.to_frame().reset_index().rename(columns = {'index': 'Region', 'Region':'Count of Highest Leisure Rate'})

In [327]:
top_ten_leisure.to_csv('TransientADR/top_ten_leisure.csv', index = False)

In [328]:
trans_rank_1 = fy_2018_ranked.loc[fy_2018_ranked['TransRank'] == 1]
trans_rank_1

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR
13,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,121534.0,67579065.0,362.98,556.05,2.0,1.0,NaN,65.28
72,2018,2,Colorado Area,39.482231,-106.046181,52475.0,20207026.0,133296.0,71685551.0,385.08,537.79,1.0,1.0,NaN,71.60
131,2018,3,Colorado Area,39.482231,-106.046181,46934.0,18417527.0,157560.0,77975202.0,392.41,494.89,1.0,1.0,NaN,79.29
196,2018,4,Florida Keys,24.778338,-80.921608,13479.0,4512416.0,72409.0,27276158.0,334.77,376.70,1.0,1.0,NaN,88.87
273,2018,5,"New York, NY",40.730610,-73.935242,213350.0,76499044.0,1040749.0,399166435.0,358.56,383.54,2.0,1.0,NaN,93.49
332,2018,6,"New York, NY",40.730610,-73.935242,207492.0,73110343.0,1005545.0,366672379.0,352.35,364.65,1.0,1.0,NaN,96.63
393,2018,7,"Oahu Island, HI",21.483521,-158.245433,40207.0,11405282.0,376278.0,122937681.0,283.66,326.72,4.0,1.0,NaN,86.82
419,2018,8,California Central Coast,35.315787,-120.857870,67371.0,22103985.0,207417.0,72424490.0,328.09,349.17,1.0,1.0,NaN,93.96
509,2018,9,"New York, NY",40.730610,-73.935242,261257.0,129983502.0,941925.0,383202930.0,497.53,406.83,1.0,1.0,NaN,122.29
568,2018,10,"New York, NY",40.730610,-73.935242,260592.0,100307576.0,1049406.0,446113388.0,384.92,425.11,1.0,1.0,NaN,90.55


In [329]:
trans_rank_1['Region'].value_counts()

New York, NY                5
Colorado Area               4
Oahu Island, HI             1
California Central Coast    1
Florida Keys                1
Name: Region, dtype: int64

In [330]:
trans_rank_1.to_csv('TransientADR/trans_rank_1.csv', index = False)

#### Lowest leisure rates

In [331]:
low_leisure_rate = fy_2018_ranked.nsmallest(12,'TransADR')
low_leisure_rate

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR
34,2018,1,"Myrtle Beach, SC",33.689060,-78.886696,28000.0,2354348.0,12651.0,1292895.0,84.08,102.20,56.0,56.0,NaN,82.27
93,2018,2,"Myrtle Beach, SC",33.689060,-78.886696,29011.0,2862726.0,21633.0,2290015.0,98.68,105.86,56.0,56.0,NaN,93.22
683,2018,12,"Myrtle Beach, SC",33.689060,-78.886696,19468.0,1632746.0,22306.0,2436703.0,83.87,109.24,57.0,57.0,NaN,76.78
28,2018,1,"Knoxville, TN",35.964668,-83.926453,12926.0,1323684.0,28763.0,3200525.0,102.40,111.27,55.0,55.0,NaN,92.03
87,2018,2,"Knoxville, TN",35.964668,-83.926453,9319.0,990735.0,33266.0,3727030.0,106.31,112.04,55.0,55.0,NaN,94.89
38,2018,1,"Norfolk/Virginia Beach, VA",36.850769,-76.285873,19217.0,2157129.0,41605.0,4713991.0,112.25,113.30,53.0,54.0,NaN,99.07
324,2018,6,"Las Vegas, NV",36.114647,-115.172813,267173.0,34601838.0,919993.0,107377471.0,129.51,116.72,57.0,58.0,NaN,110.96
663,2018,12,"Colorado Springs, CO",38.846127,-104.800644,7877.0,896504.0,22253.0,2599979.0,113.81,116.84,53.0,56.0,NaN,97.41
442,2018,8,"Las Vegas, NV",36.114647,-115.172813,184446.0,21331567.0,1020353.0,119655328.0,115.65,117.27,58.0,58.0,NaN,98.62
383,2018,7,"Las Vegas, NV",36.114647,-115.172813,269848.0,35471461.0,995081.0,117954886.0,131.45,118.54,54.0,58.0,NaN,110.89


In [332]:
low_leisure_rate['Counts'] = low_leisure_rate.groupby(['Region'])['TransADR'].transform('count')
low_leisure_rate

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR,Counts
34,2018,1,"Myrtle Beach, SC",33.689060,-78.886696,28000.0,2354348.0,12651.0,1292895.0,84.08,102.20,56.0,56.0,NaN,82.27,4
93,2018,2,"Myrtle Beach, SC",33.689060,-78.886696,29011.0,2862726.0,21633.0,2290015.0,98.68,105.86,56.0,56.0,NaN,93.22,4
683,2018,12,"Myrtle Beach, SC",33.689060,-78.886696,19468.0,1632746.0,22306.0,2436703.0,83.87,109.24,57.0,57.0,NaN,76.78,4
28,2018,1,"Knoxville, TN",35.964668,-83.926453,12926.0,1323684.0,28763.0,3200525.0,102.40,111.27,55.0,55.0,NaN,92.03,2
87,2018,2,"Knoxville, TN",35.964668,-83.926453,9319.0,990735.0,33266.0,3727030.0,106.31,112.04,55.0,55.0,NaN,94.89,2
38,2018,1,"Norfolk/Virginia Beach, VA",36.850769,-76.285873,19217.0,2157129.0,41605.0,4713991.0,112.25,113.30,53.0,54.0,NaN,99.07,1
324,2018,6,"Las Vegas, NV",36.114647,-115.172813,267173.0,34601838.0,919993.0,107377471.0,129.51,116.72,57.0,58.0,NaN,110.96,3
663,2018,12,"Colorado Springs, CO",38.846127,-104.800644,7877.0,896504.0,22253.0,2599979.0,113.81,116.84,53.0,56.0,NaN,97.41,2
442,2018,8,"Las Vegas, NV",36.114647,-115.172813,184446.0,21331567.0,1020353.0,119655328.0,115.65,117.27,58.0,58.0,NaN,98.62,3
383,2018,7,"Las Vegas, NV",36.114647,-115.172813,269848.0,35471461.0,995081.0,117954886.0,131.45,118.54,54.0,58.0,NaN,110.89,3


In [333]:
low_leisure_rate['Region'].value_counts()

Myrtle Beach, SC              4
Las Vegas, NV                 3
Colorado Springs, CO          2
Knoxville, TN                 2
Norfolk/Virginia Beach, VA    1
Name: Region, dtype: int64

In [334]:
low_leisure_rate.to_csv('TransientADR/low_leisure_rate.csv', index = False)

#### Lowest group rates

In [335]:
low_group_rate = fy_2018_ranked.nsmallest(12,'GroupADR')
low_group_rate

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR
683,2018,12,"Myrtle Beach, SC",33.689060,-78.886696,19468.0,1632746.0,22306.0,2436703.0,83.87,109.24,57.0,57.0,NaN,76.78
34,2018,1,"Myrtle Beach, SC",33.689060,-78.886696,28000.0,2354348.0,12651.0,1292895.0,84.08,102.20,56.0,56.0,NaN,82.27
624,2018,11,"Myrtle Beach, SC",33.689060,-78.886696,33766.0,3268099.0,25159.0,3041992.0,96.79,120.91,57.0,57.0,NaN,80.05
93,2018,2,"Myrtle Beach, SC",33.689060,-78.886696,29011.0,2862726.0,21633.0,2290015.0,98.68,105.86,56.0,56.0,NaN,93.22
28,2018,1,"Knoxville, TN",35.964668,-83.926453,12926.0,1323684.0,28763.0,3200525.0,102.40,111.27,55.0,55.0,NaN,92.03
146,2018,3,"Knoxville, TN",35.964668,-83.926453,14932.0,1531326.0,51170.0,6572125.0,102.55,128.44,58.0,57.0,NaN,79.84
87,2018,2,"Knoxville, TN",35.964668,-83.926453,9319.0,990735.0,33266.0,3727030.0,106.31,112.04,55.0,55.0,NaN,94.89
677,2018,12,"Knoxville, TN",35.964668,-83.926453,9812.0,1062650.0,57837.0,9196269.0,108.30,159.00,56.0,38.0,NaN,68.11
678,2018,12,"Las Vegas, NV",36.114647,-115.172813,109036.0,11973458.0,678564.0,83544938.0,109.81,123.12,55.0,55.0,NaN,89.19
205,2018,4,"Knoxville, TN",35.964668,-83.926453,18636.0,2061982.0,46017.0,6339970.0,110.65,137.77,58.0,58.0,NaN,80.32


In [336]:
low_group_rate['Counts'] = low_group_rate.groupby(['Region'])['TransADR'].transform('count')
low_group_rate

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,TransientDemandTY,TransientRevenueTY,GroupADR,TransADR,GroupRank,TransRank,ADR Correl,GroupADR Ratio TransADR,Counts
683,2018,12,"Myrtle Beach, SC",33.689060,-78.886696,19468.0,1632746.0,22306.0,2436703.0,83.87,109.24,57.0,57.0,NaN,76.78,4
34,2018,1,"Myrtle Beach, SC",33.689060,-78.886696,28000.0,2354348.0,12651.0,1292895.0,84.08,102.20,56.0,56.0,NaN,82.27,4
624,2018,11,"Myrtle Beach, SC",33.689060,-78.886696,33766.0,3268099.0,25159.0,3041992.0,96.79,120.91,57.0,57.0,NaN,80.05,4
93,2018,2,"Myrtle Beach, SC",33.689060,-78.886696,29011.0,2862726.0,21633.0,2290015.0,98.68,105.86,56.0,56.0,NaN,93.22,4
28,2018,1,"Knoxville, TN",35.964668,-83.926453,12926.0,1323684.0,28763.0,3200525.0,102.40,111.27,55.0,55.0,NaN,92.03,5
146,2018,3,"Knoxville, TN",35.964668,-83.926453,14932.0,1531326.0,51170.0,6572125.0,102.55,128.44,58.0,57.0,NaN,79.84,5
87,2018,2,"Knoxville, TN",35.964668,-83.926453,9319.0,990735.0,33266.0,3727030.0,106.31,112.04,55.0,55.0,NaN,94.89,5
677,2018,12,"Knoxville, TN",35.964668,-83.926453,9812.0,1062650.0,57837.0,9196269.0,108.30,159.00,56.0,38.0,NaN,68.11,5
678,2018,12,"Las Vegas, NV",36.114647,-115.172813,109036.0,11973458.0,678564.0,83544938.0,109.81,123.12,55.0,55.0,NaN,89.19,1
205,2018,4,"Knoxville, TN",35.964668,-83.926453,18636.0,2061982.0,46017.0,6339970.0,110.65,137.77,58.0,58.0,NaN,80.32,5


In [337]:
low_group_rate['Region'].value_counts()

Knoxville, TN                 5
Myrtle Beach, SC              4
Las Vegas, NV                 1
St Louis, MO-IL               1
Norfolk/Virginia Beach, VA    1
Name: Region, dtype: int64

In [338]:
low_group_rate.to_csv('TransientADR/low_group_rate.csv', index = False)